In [1]:
MODEL_NAME = 'first_try'

In [2]:
import pandas as pd
import os

def get_image_path(image_id:int):
    return os.path.join('../tiles_768', str(image_id))

train = pd.read_csv(f"../data/train.csv")

train['tile_path'] = train['image_id'].apply(lambda x: get_image_path(x))
train.head()

,image_id,label,image_width,image_height,is_tma,tile_path
0,4,HGSC,23785,20008,False,../tiles_768/4
1,66,LGSC,48871,48195,False,../tiles_768/66
2,91,HGSC,3388,3388,True,../tiles_768/91
3,281,LGSC,42309,15545,False,../tiles_768/281
4,286,EC,37204,30020,False,../tiles_768/286


In [3]:
from PIL import Image
import torch
import torch.nn as nn
import timm
from timm.models.layers import DropPath
import copy
from itertools import cycle

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "timm/coatnet_rmlp_2_rw_384.sw_in12k_ft_in1k"

print(f"Using device {device} and model {model_name}")

model = timm.create_model(model_name, pretrained=True)

drop_path_rate = 0.3
dropout_rate = 0.1
drop_path_rates = []
total_blocks = 0
for stage in model.stages:
    if hasattr(stage, 'blocks'):
        for block in stage.blocks:
            total_blocks += 1
drop_path_rates = [x.item() for x in torch.linspace(0, drop_path_rate, total_blocks)]
        
i_drop = 0
# Assign drop path rates
for stage in model.stages:
    if hasattr(stage, 'blocks'):
        for block in stage.blocks:
            block.drop_path = DropPath(drop_prob=drop_path_rates[i_drop])
            block.drop_path1 = DropPath(drop_prob=drop_path_rates[i_drop])
            block.drop_path2 = DropPath(drop_prob=drop_path_rates[i_drop])
            if hasattr(block, 'attn'):
                block.attn.attn_drop = nn.Dropout(p=dropout_rate, inplace=False)
                block.attn.proj_drop = nn.Dropout(p=dropout_rate, inplace=False)
            if hasattr(block, 'mlp'):
                block.mlp.drop1 = nn.Dropout(p=dropout_rate, inplace=False)
                block.mlp.drop2 = nn.Dropout(p=dropout_rate, inplace=False)
            
            i_drop += 1

model.head.drop = nn.Dropout(p=dropout_rate, inplace=False)
model.head.fc = nn.Linear(model.head.in_features, 5)

model = model.to(device)

# Initialize EMA model
ema_decay = 0.999  # decay factor for EMA
ema_model = copy.deepcopy(model)
ema_model = ema_model.to(device)

Using device cuda and model timm/coatnet_rmlp_2_rw_384.sw_in12k_ft_in1k


/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
import os
from PIL import Image
from torch.utils.data import Dataset
import random

integer_to_label = {
    0: 'HGSC',
    1: 'CC',
    2: 'EC',
    3: 'LGSC',
    4: 'MC',
}

label_to_integer = {
    'HGSC': 0,
    'CC': 1,
    'EC': 2,
    'LGSC': 3,
    'MC': 4,
}
    
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.all_images = []  # Store all images in an interlaced fashion
        self.folder_images = []  # Temporary storage for images from each folder

        # Step 1: Collect all images from each folder
        for index, row in dataframe.iterrows():
            folder_path = row['tile_path']
            label = row['label']
            image_id = row['image_id']
            if os.path.isdir(folder_path):
                image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                random.shuffle(image_files)
                self.folder_images.append([(image_file, label, image_id) for image_file in image_files])

        # Step 2: Interlace the images
        max_length = max(len(images) for images in self.folder_images)
        for i in range(max_length):
            for images in self.folder_images:
                self.all_images.append(images[i % len(images)])

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_path, label, image_id = self.all_images[idx]
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label_to_integer[label], image_id

In [5]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms

BATCH_SIZE = 8

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=384, scale=(0.75, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomAffine(degrees=0, shear=(-20, 20, -20, 20)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.3, hue=0.3),
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.25),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1))], p=0.25),
    transforms.Resize(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.5,
        0.5,
        0.5
    ], std=[
        0.5,
        0.5,
        0.5
    ]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

val_transform = transforms.Compose([
    transforms.Resize(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.5,
        0.5,
        0.5
    ], std=[
        0.5,
        0.5,
        0.5
    ]),
])

train_dataset = ImageDataset(dataframe=train, transform=train_transform)

# Calculate weights for each class
class_counts = [train.groupby('label').count().loc[label]['image_id'] for label in label_to_integer]  # Example class counts
num_samples = sum(class_counts)
class_weights = [num_samples / class_count for class_count in class_counts]

# Assign a weight to each sample in the dataset based on its class
sample_weights = [class_weights[label_to_integer[label]] for _, label, _ in train_dataset.all_images]

# Create WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader with WeightedRandomSampler
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=3)

In [6]:
import logging
import sys

# Get the root logger
logger = logging.getLogger()

# Optional: Remove all existing handlers from the logger
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a FileHandler and add it to the logger
file_handler = logging.FileHandler(f'logs/coatnet_train_{MODEL_NAME}.txt')
file_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)

# Create a StreamHandler for stderr and add it to the logger
stream_handler = logging.StreamHandler(sys.stderr)
stream_handler.setLevel(logging.ERROR)  # Only log ERROR and CRITICAL messages to stderr
logger.addHandler(stream_handler)

In [7]:
import torch
import torch.optim as optim
import logging
import numpy as np
import math
from sklearn.metrics import balanced_accuracy_score
import random
from torch.cuda.amp import GradScaler, autocast

initial_lr = 0.0005 * BATCH_SIZE/256
final_lr = initial_lr * 0.01
num_epochs = 10000

# Function for linear warmup
def learning_rate(step, warmup_steps=2500, max_steps=25000):
    if step < warmup_steps:
        return initial_lr * (float(step) / float(max(1, warmup_steps)))
    elif step < max_steps:
        progress = (float(step - warmup_steps) / float(max(1, max_steps - warmup_steps)))
        cos_component = 0.5 * (1 + math.cos(math.pi * progress))
        return final_lr + (initial_lr - final_lr) * cos_component
    else:
        return final_lr

def update_ema_variables(model, ema_model, alpha):
    # Update the EMA model parameters
    with torch.no_grad():
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(alpha).add_(param.data, alpha=1 - alpha)

scaler = GradScaler()
optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=5e-2)

# Define the loss function with class weights
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

best_val_accuracy = 0.0
step = 0

for epoch in range(num_epochs):
    model.train()  # set the model to training mode
    
    for i, (images, labels, _) in enumerate(train_dataloader, 0):
        # Convert images to PIL format
        images = images.to(device)
        labels = labels.to(device)
        
        # Linearly increase the learning rate
        lr = learning_rate(step)
        for g in optimizer.param_groups:
            g['lr'] = lr

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass with autocast
        with autocast():
            outputs = model(images)
            logits_per_image = outputs
            loss = criterion(logits_per_image, labels)
        
        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        update_ema_variables(model, ema_model, ema_decay)

        logging.info('[%d, %5d] loss: %.3f' % (epoch + 1, step, loss.item()))

        if step % 2500 == 0:
            ema_model.eval()
            torch.save(ema_model.state_dict(), f'coatnet_models_{MODEL_NAME}/epoch_{epoch}_step_{step}.pth')
            logging.info(f'Model saved after epoch {epoch} and step {step}')\

            model.train()

        if step == 25000:
            torch.save(ema_model.state_dict(), f'coatnet_models_{MODEL_NAME}/final.pth')
            break
    
        step += 1
        
    if step >= 25000:
        break

OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB (GPU 0; 7.43 GiB total capacity; 6.60 GiB already allocated; 26.00 MiB free; 6.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import shutil
import os

def duplicate_ipynb_with_new_name(src_file_path, dest_dir, new_name):
    """
    Duplicate an IPython notebook file to a new directory with a new file name.

    Parameters:
    src_file_path (str): The path of the source IPython notebook file.
    dest_dir (str): The destination directory where the file will be copied.
    new_name (str): The new file name for the duplicated notebook.

    Returns:
    str: The path of the duplicated file with the new name.
    """
    # Check if the new name contains the '.ipynb' extension, add if not
    if not new_name.endswith('.ipynb'):
        new_name += '.ipynb'

    # Creating the destination file path with the new name
    dest_file_path = os.path.join(dest_dir, new_name)

    # Copying the file to the new directory
    shutil.copy(src_file_path, dest_file_path)

    return dest_file_path

src_file = "eva02-base-finetune.ipynb"
dest_directory = "notebook_history"
new_filename = f"{MODEL_NAME}.ipynb"
duplicate_ipynb_with_new_name(src_file, dest_directory, new_filename)


In [ ]:
import random
from sklearn.metrics import balanced_accuracy_score

ema_model.eval()

image_ids = []
logits = []
labels = []

with torch.no_grad():
    for _, row in train.iterrows():
        path = row['tile_path']
        all_files = [f for f in os.listdir(path) if f.lower().endswith('.png')]

        sum_probabilities = torch.zeros(5).to(device)
        sum_log_probabilities = torch.zeros(5).to(device)
        sum_log_neg_probabilities = torch.zeros(5).to(device)
        
        batch_logits = []

        # Prepare a list to hold image tiles
        batch_tiles = []

        sample_size = min(128, len(all_files))
        sampled_files = random.sample(all_files, sample_size)

        for image_name in sampled_files:
            image_path = os.path.join(path, image_name)
            sub_image = Image.open(image_path)

            tile = val_transform(sub_image).unsqueeze(0).to(device)
            batch_tiles.append(tile)
        
        for i_batch in range(0, len(batch_tiles), 32):
            outputs = ema_model(torch.concat(batch_tiles[i_batch:i_batch+32], dim=0))
            probs = outputs.softmax(dim=1)
            batch_logits.append(outputs)
            sum_probabilities += probs.sum(dim=0)
            sum_log_probabilities += torch.log(probs).sum(dim=0)
            sum_log_neg_probabilities += torch.log(1 - probs).sum(dim=0)
        
        image_id = row['image_id']
        batch_logits = torch.concat(batch_logits, dim=0)
        label = row['label']
        print(batch_logits.shape, label, image_id, integer_to_label[sum_probabilities.argmax().cpu().item()], integer_to_label[sum_log_probabilities.argmax().cpu().item()], integer_to_label[sum_log_neg_probabilities.argmin().cpu().item()])
        image_ids.append(image_id)
        logits.append(batch_logits)
        labels.append(label)
        

In [ ]:
predictions = []
for image_logits in logits:
    argmax_indices = torch.argmax(image_logits, dim=1)

    frequency_counts = torch.bincount(argmax_indices, minlength=5)

    max_vote_key = frequency_counts.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

plurality_accuracy = balanced_accuracy_score(labels, predictions)
plurality_accuracy

In [ ]:
predictions = []
for image_logits in logits:
    summed_logits = image_logits.sum(dim=0)
    
    max_vote_key = summed_logits.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

logit_sum_accuracy = balanced_accuracy_score(labels, predictions)
logit_sum_accuracy

In [ ]:
predictions = []
for image_logits in logits:
    summed_probs = image_logits.softmax(dim=1).sum(dim=0)
    
    max_vote_key = summed_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
prob_sum_accuracy

In [ ]:
predictions = []
for image_logits in logits:
    summed_log_probs = torch.log(image_logits.softmax(dim=1)).sum(dim=0)
    
    max_vote_key = summed_log_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
log_prob_sum_accuracy

In [ ]:
predictions = []
for image_logits in logits:
    summed_one_minus_log_probs = torch.log(1 - image_logits.softmax(dim=1)).sum(dim=0)
    
    min_vote_key = summed_one_minus_log_probs.argmin().cpu().item()
    predictions.append(integer_to_label[min_vote_key])

one_minus_log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
one_minus_log_prob_sum_accuracy

In [ ]:
predictions = []
for i, image_logits in enumerate(logits):
    summed_log_probs = torch.log(image_logits.softmax(dim=1)).sum(dim=0)
    summed_one_minus_log_probs = torch.log(1 - image_logits.softmax(dim=1)).sum(dim=0)
    if summed_log_probs.argmax().cpu().item() != summed_one_minus_log_probs.argmin().cpu().item():
        print(i, labels[i], integer_to_label[summed_log_probs.argmax().cpu().item()], integer_to_label[summed_one_minus_log_probs.argmin().cpu().item()], summed_log_probs, summed_one_minus_log_probs)


In [ ]:
from collections import defaultdict

def borda_count_winner(image_logits):
    num_classes = image_logits.size(1)
    borda_scores = defaultdict(int)

    # Rank each class for each image_logits and assign Borda points
    for logits in image_logits:
        # Get ranks (in descending order of logits)
        ranks = torch.argsort(logits, descending=True)

        # Assign Borda points (highest rank gets num_classes - 1 points, next gets num_classes - 2, ...)
        for rank, class_index in enumerate(ranks):
            borda_scores[class_index.item()] += num_classes - 1 - rank

    # Find the class with the highest total Borda score
    borda_winner = max(borda_scores, key=borda_scores.get)
    return borda_winner

predictions = []
for image_logits in logits:
    borda_winner = borda_count_winner(image_logits)
    predictions.append(integer_to_label[borda_winner])
    
borda_accuracy = balanced_accuracy_score(labels, predictions)
borda_accuracy

In [ ]:
def instant_runoff_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    active_candidates = set(range(num_classes))

    while True:
        # Count the first-preference votes for each candidate
        first_pref_counts = torch.zeros(num_classes)
        for logits in image_logits:
            for rank in torch.argsort(logits, descending=True):
                if rank.item() in active_candidates:
                    first_pref_counts[rank] += 1
                    break

        # Check if any candidate has more than 50% of the votes
        if torch.any(first_pref_counts > num_voters / 2):
            return torch.argmax(first_pref_counts).item()

        # Find the candidate with the fewest votes among active candidates
        active_candidates_votes = first_pref_counts[list(active_candidates)]
        min_votes, min_index = torch.min(active_candidates_votes, 0)
        min_candidate = list(active_candidates)[min_index.item()]

        # Eliminate the candidate with the fewest votes
        active_candidates.remove(min_candidate)
        
predictions = []
for image_logits in logits:
    irv_winner = instant_runoff_winner(image_logits)
    predictions.append(integer_to_label[irv_winner])
    
instant_runoff_accuracy = balanced_accuracy_score(labels, predictions)
instant_runoff_accuracy

In [ ]:
def calculate_minimax_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    max_regrets = torch.zeros(num_classes)

    # Perform pairwise comparisons between all classes
    for i in range(num_classes):
        for j in range(i + 1, num_classes):
            # Count how many voters prefer class i over class j and vice versa
            votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
            votes_for_j = num_voters - votes_for_i

            # Update the maximum regret for each class
            max_regrets[i] = max(max_regrets[i], votes_for_j)
            max_regrets[j] = max(max_regrets[j], votes_for_i)

    # The winner is the class with the smallest maximum regret
    return torch.argmin(max_regrets).item()
        
predictions = []
for image_logits in logits:
    minimax_winner = calculate_minimax_winner(image_logits)
    predictions.append(integer_to_label[minimax_winner])
    
minimax_accuracy = balanced_accuracy_score(labels, predictions)
minimax_accuracy

In [ ]:
from collections import defaultdict

def find_winner(graph, source, visited):
    """Helper function to find the winner in the graph."""
    if source not in graph:
        return False

    visited.add(source)
    for target in graph[source]:
        if target not in visited and find_winner(graph, target, visited):
            return True
    visited.remove(source)
    return False

def ranked_pairs_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    margins = defaultdict(int)

    # Perform pairwise comparisons
    for i in range(num_classes):
        for j in range(num_classes):
            if i != j:
                votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
                votes_for_j = num_voters - votes_for_i
                margins[(i, j)] = votes_for_i - votes_for_j

    # Sort pairs by margin of victory
    sorted_pairs = sorted(margins, key=margins.get, reverse=True)

    # Initialize graph for locking pairs
    graph = {i: set() for i in range(num_classes)}
    for pair in sorted_pairs:
        winner, loser = pair
        graph[winner].add(loser)
        visited = set()

        # Check for cycle
        if find_winner(graph, loser, visited):
            graph[winner].remove(loser)

    # Determine the winner
    for i in range(num_classes):
        if not any(i in targets for targets in graph.values()):
            return i

    # Fallback: Choose the class with the highest total votes if no clear winner is found
    total_votes = torch.sum(image_logits, axis=0)
    return torch.argmax(total_votes).item()
        
predictions = []
for image_logits in logits:
    rp_winner = ranked_pairs_winner(image_logits)
    predictions.append(integer_to_label[rp_winner])
    
rp_accuracy = balanced_accuracy_score(labels, predictions)
rp_accuracy

In [ ]:
from collections import defaultdict

def star_voting_winner(image_logits):
    num_voters, num_classes = image_logits.shape

    # Step 1: Sum the scores for each candidate
    total_scores = torch.sum(image_logits, axis=0)

    # Step 2: Find the two candidates with the highest total scores
    top_two = torch.topk(total_scores, 2).indices

    # Step 3: Runoff between the top two candidates
    first_choice_votes = torch.sum(image_logits[:, top_two[0]] > image_logits[:, top_two[1]])
    second_choice_votes = num_voters - first_choice_votes

    # Determine the winner
    if first_choice_votes > second_choice_votes:
        return top_two[0].item()
    else:
        return top_two[1].item()
        
predictions = []
for image_logits in logits:
    star_winner = ranked_pairs_winner(image_logits)
    predictions.append(integer_to_label[star_winner])
    
star_accuracy = balanced_accuracy_score(labels, predictions)
star_accuracy